In [1]:
import os
from os import path
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

In [2]:
out_path = '../hint/out/'
data_path = '../data/'
data_files = 'PANcancer_expression_data/'
files = glob(data_path+data_files+'*/[!MANIFEST]*.txt')

In [3]:
cancer_type = ['LUSC','HNSC','BLCA', 'GBM', 'LUAD', 'KIRC', 'UCEC', 'COADREAD', 'BRCA']

In [ ]:
df = pd.read_csv(files[0], delimiter = '\t', low_memory=False)
df.drop(df.index[0], inplace=True)

In [32]:
df.shape

(20531, 553)

In [155]:
df.head()

,Hybridization REF,TCGA-18-3406-01A-01R-0980-07,TCGA-18-3407-01A-01R-0980-07,TCGA-18-3408-01A-01R-0980-07,TCGA-18-3409-01A-01R-0980-07,TCGA-18-3410-01A-01R-0980-07,TCGA-18-3411-01A-01R-0980-07,TCGA-18-3412-01A-01R-0980-07,TCGA-18-3414-01A-01R-0980-07,TCGA-18-3415-01A-01R-0980-07,...,TCGA-NK-A5CX-01A-11R-A26W-07,TCGA-NK-A5D1-01A-11R-A26W-07,TCGA-NK-A7XE-01A-12R-A405-07,TCGA-O2-A52N-01A-11R-A26W-07,TCGA-O2-A52Q-01A-11R-A26W-07,TCGA-O2-A52S-01A-11R-A262-07,TCGA-O2-A52V-01A-31R-A262-07,TCGA-O2-A52W-01A-11R-A26W-07,TCGA-O2-A5IB-01A-11R-A27Q-07,TCGA-XC-AA0X-01A-32R-A405-07
1,?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,?|100133144,75.3622,14.4148,8.4888,2.5897,16.6653,12.9418,47.5782,12.3168,55.1387,...,40.3766,39.5911,20.8723,30.2240,4.2895,19.9503,30.0872,53.6593,72.5666,44.7501
3,?|100134869,25.4252,6.9933,3.3754,7.1538,19.5722,14.4142,20.8881,25.7116,39.0809,...,17.3322,21.7407,15.7901,4.2022,3.6632,47.1026,15.2957,33.3907,42.1832,23.6304
4,?|10357,279.9764,198.2540,254.5929,164.3744,200.8033,277.6667,394.3623,205.8704,162.0405,...,142.3695,234.1647,133.6915,131.5628,153.5980,176.7177,188.7215,260.3332,160.1624,76.6488
5,?|10431,1510.2362,968.6013,1474.1213,944.6154,1156.0568,1119.5816,861.5023,1030.4597,1931.2139,...,935.5667,777.4533,2257.2693,607.6503,1103.4483,1188.3278,1248.0303,789.3606,460.8626,805.9507


In [238]:
# Merge all the datasets that are part of cancer type list
patients_list = []
patients_list.append(df.columns.values[1:])
for file in files[1:]:
    
    type_ = file.split('/')[-1].split('.')[0] 
    if not type_ in cancer_type: 
        print('Skipped cancer ', type_)
        continue
        
    df1 = pd.read_csv(file, delimiter = '\t', low_memory=False)
    df1.drop(df1.index[0], inplace=True)
    patients_list.append(df1.columns.values[1:])
    df1.drop(df1.columns[0], axis=1 ,inplace=True)
    df = pd.concat([df,df1],axis=0, sort=False)

Skipped cancer  LAML
Skipped cancer  OV


In [243]:
df_ = df.T

In [245]:
df_.index = np.concatenate((np.array(['genes',]),df_.index[1:].values))
df_.columns = df_.iloc[0].values
df_.drop(df_.index[0], inplace = True )

In [246]:
genes = df_.columns.values

In [247]:
df_ = df_.drop(df_.columns[df_.apply(lambda col: '?' in str(col))], axis=1)
df_.columns = map(lambda col: str(col).split('|')[0], df_.columns.values)

In [248]:
df_['y'] = [0 if int(str(p).split('-')[3][:2]) > 10 else 1 for p in df_.index.values]

In [254]:
df_.to_csv(data_path+'pancancer_all_subtypes.csv')

In [ ]:
df = pd.read_csv(data_path+'pancancer_all_subtypes.csv')

In [253]:
np.unique(df_['y'])

array([0, 1])